In this notebook we are going to see how to convert speech into text using Facebook Wav2Vec 2.0 model.Wav2Vec2 is a speech model that accepts a float array corresponding to the raw waveform of the speech signal. Wav2Vec2 model was trained using connectionist temporal classification (CTC) so the model output has to be decoded using Wav2Vec2Tokenizer.For learning more about it click on this [link](https://huggingface.co/transformers/model_doc/wav2vec2.html)

In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 132.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
import transformers
print(transformers.__version__)

4.57.0


If you don't see at least 4.3.0 version,then upgrade it

### Import Libraries

In [ ]:
import librosa
import torch
import IPython.display as display
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import numpy as np

### Load pre-trained Wav2Vec model

In [ ]:
#load pre-trained model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.12/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load Audio file

In [ ]:
#load audio file
# Replace '/content/audiofile.m4a' with the actual path if you uploaded it to a different location
audio, sampling_rate = librosa.load('/content/audiofile.m4a', sr=16000)

/tmp/ipython-input-669518037.py:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sampling_rate = librosa.load('/content/audiofile.m4a', sr=16000)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
audio,sampling_rate

(array([-1.38280475e-05, -2.14298852e-05,  2.24593077e-06, ...,
         5.63148569e-06, -5.96416794e-06,  0.00000000e+00], dtype=float32),
 16000)

# Play the Audio

In [ ]:
# audio
display.Audio("/content/audiofile.m4a", autoplay=True)

### Speech to Text

First of all tokenize the input values,take the maximum prediction from the logit and then extraxt the text

In [ ]:
input_values = tokenizer(audio, return_tensors = 'pt').input_values
input_values

tensor([[ 4.7878e-06, -2.4991e-04,  5.4335e-04,  ...,  6.5678e-04,
          2.6827e-04,  4.6810e-04]])

In [ ]:
# store logits (non-normalized predictions)
logits = model(input_values).logits
logits

tensor([[[ 13.5454, -26.9272, -26.6922,  ...,  -6.3552,  -6.4504,  -7.6013],
         [ 13.5672, -27.0129, -26.7786,  ...,  -6.3625,  -6.4637,  -7.5776],
         [ 14.0376, -27.2252, -26.9936,  ...,  -6.2225,  -6.3798,  -7.7601],
         ...,
         [  9.5905, -22.7072, -22.4734,  ...,  -5.6545,  -6.1188,  -7.3590],
         [  9.9305, -22.5057, -22.2692,  ...,  -5.7600,  -6.2040,  -6.5669],
         [ 10.8207, -22.8730, -22.6375,  ...,  -5.5668,  -5.9146,  -6.4621]]],
       grad_fn=<ViewBackward0>)

In [ ]:
# store predicted id's
# pass the logit values to softmax to get the predicted values
predicted_ids = torch.argmax(logits, dim =-1)

In [ ]:
# pass the prediction to the tokenzer decode to get the transcription
transcriptions = tokenizer.decode(predicted_ids[0])

In [ ]:
transcriptions

'DO WE WANT WHAT DO YOU WANT'